In [1]:
import gspread, gspread_dataframe
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials

import pandas as pd
import re

def num_to_col_letters(num) :
    letters = ''
    while num:
        mod = (num - 1) % 26
        letters += chr(mod + 65)
        num = (num - 1) // 26
    return ''.join(reversed(letters))

In [2]:
def connect_to_sheets(spreadsheet='GemeinsameBilanzierung_16_17', sheetname='') :
    
    if type(spreadsheet) is str : 
        scope = ['https://spreadsheets.google.com/feeds']
        creds = ServiceAccountCredentials.from_json_keyfile_name('client-secret.json', scope)
        client = gspread.authorize(creds)
        spreadsheet = client.open(spreadsheet)

    # Find a workbook by name and open the first sheet
    # Make sure you use the right name here.
    if sheetname == '' :
        return spreadsheet
    else :
        worksheet = spreadsheet.worksheet(sheetname)
        return worksheet

In [3]:
workbook = connect_to_sheets()
sheet = connect_to_sheets(workbook, 'August')

In [12]:
col_titles = ['description', 'date', 'amount']

budgeting = get_as_dataframe(sheet, parse_dates=True, usecols=range(4), 
                             skiprows=5, header=None).dropna(how='all')
budgeting.columns = ['budget_type'] + col_titles

max_bargeld = get_as_dataframe(sheet, parse_dates=True, usecols=range(7,10), 
                               skiprows=6, header=None).dropna(how='all')
max_bargeld.columns = col_titles
max_bargeld['money_pot'] = 'BM'

paul_bargeld = get_as_dataframe(sheet, parse_dates=True, usecols=range(10,13), 
                                skiprows=6, header=None).dropna(how='all')
paul_bargeld.columns = col_titles
paul_bargeld['money_pot'] = 'BP'

konto = get_as_dataframe(sheet, parse_dates=True, usecols=range(13,16), 
                         skiprows=6, header=None).dropna(how='all')
konto.columns = col_titles
konto['money_pot'] = 'KG'

conjoined = pd.concat([max_bargeld, paul_bargeld, konto])

In [15]:
all_info = pd.merge(budgeting, conjoined, how='outer', 
                    on=['description', 'date', 'amount'], indicator=True)

right_only = all_info[all_info['_merge'] == 'right_only']

display(right_only)

,budget_type,description,date,amount,money_pot,_merge
113,NaN,Geld aufladen Mitarbeiter ausweis,01/08/17,-20,BM,right_only
114,NaN,Geld abheben,03/08/17,30,BM,right_only
115,NaN,Geld aufladen Trianon,02/08/17,-20,BM,right_only
116,NaN,Geld aus Schatulle,05/08/17,"1,3",BM,right_only
117,NaN,Betriebsausflug,07/08/17,"-18,5",BM,right_only
118,NaN,Geldfund,12/08/17,"0,05",BM,right_only
119,NaN,BBk Karte Aufladen,14/08/17,-10,BM,right_only
120,NaN,Geld aus Schatulle,20/08/17,"0,5",BM,right_only
121,NaN,Schatullenkorrektur,20/08/17,"=2,17-1,12",BM,right_only
122,NaN,Schatullenkorrektur,20/08/17,"-1,05",BM,right_only


In [28]:
def simple_payment(date, description, amount, budget_type, transaction_type, money_pot): 
    new_id = id_generation(date)
    crsr = db.cursor()

    crsr.execute("INSERT INTO money_events VALUES ({}, '{}', '{}', DATE('{}'), NULL, NULL, 'auto-gen');".format(
        new_id, transaction_type, description, date))
    crsr.execute("INSERT INTO budget_events VALUES ({}, '{}', {}, NULL, DATE('{}'), NULL, NULL, 'auto-gen');".format(
        new_id, budget_type, amount, date))
    crsr.execute("INSERT INTO payments VALUES ({}, '{}', {}, NULL, NULL, DATE('{}'), NULL, NULL, 'auto-gen');".format(
        new_id, money_pot, amount, date))

    db.commit()

In [29]:
for index, row in perfect_result.iterrows() :
    simple_payment(row['date'], row['description'], row['amount'], row['budget_type'], 
                   'Kartenzahlung' if row['money_pot'] == 'KG' else 'Barzahlung', row['money_pot'])

2017-08-01 00:00:00
2017-08-02 00:00:00
2017-08-01 00:00:00
2017-08-01 00:00:00
2017-08-10 00:00:00
2017-08-03 00:00:00
2017-07-31 00:00:00
2017-07-31 00:00:00
2017-07-31 00:00:00
2017-08-01 00:00:00
2017-08-01 00:00:00
2017-08-01 00:00:00
2017-08-01 00:00:00
2017-08-01 00:00:00
2017-08-02 00:00:00
2017-08-04 00:00:00
2017-08-05 00:00:00
2017-08-05 00:00:00
2017-08-05 00:00:00
2017-08-05 00:00:00
2017-08-06 00:00:00
2017-08-06 00:00:00
2017-08-07 00:00:00
2017-08-07 00:00:00
2017-08-07 00:00:00
2017-08-07 00:00:00
2017-08-07 00:00:00
2017-08-06 00:00:00
2017-08-13 00:00:00
2017-08-13 00:00:00
2017-08-11 00:00:00
2017-08-11 00:00:00
2017-08-13 00:00:00


In [30]:
def print_action_tables() :
    crsr = db.cursor()

    for table in ['money_events', 'budget_events', 'payments'] :
        display(pd.read_sql_query('SELECT * FROM {};'.format(table), db))
    
print_action_tables()

,id,type,description,creation_date,modification_dates,comments,complete
0,20170732,Kartenzahlung,Aldi,2017-07-31,None,None,YES
1,20170733,Kartenzahlung,Monatskarte FFM,2017-07-31,None,None,YES
2,20170734,Kartenzahlung,Netto,2017-07-31,None,None,YES
3,20170802,Kartenzahlung,Miete,2017-08-01,None,None,YES
4,20170803,Kartenzahlung,Miete FFM,2017-08-02,None,None,YES
5,20170804,Kartenzahlung,Berufsunfähigkeitsversicherung,2017-08-01,None,None,YES
6,20170805,Kartenzahlung,Haftpflichtversicherung,2017-08-01,None,None,YES
7,20170806,Kartenzahlung,Fitnessstudio,2017-08-03,None,None,YES
8,20170807,Kartenzahlung,Rewe,2017-08-01,None,None,YES
9,20170808,Kartenzahlung,Rossmann,2017-08-01,None,None,YES


,id,budget_pot,amount,additional_description,creation_date,modification_dates,comments,complete
0,20170802,R,-568.00,None,2017-08-01,None,None,YES
1,20170803,R,-450.00,None,2017-08-02,None,None,YES
2,20170804,R,-49.05,None,2017-08-01,None,None,YES
3,20170805,R,-7.50,None,2017-08-01,None,None,YES
4,20170811,R,-7.99,None,2017-08-10,None,None,YES
5,20170806,G,-39.80,None,2017-08-03,None,None,YES
6,20170732,L,-23.97,None,2017-07-31,None,None,YES
7,20170733,R,-87.40,None,2017-07-31,None,None,YES
8,20170734,L,-9.44,None,2017-07-31,None,None,YES
9,20170807,L,-8.71,None,2017-08-01,None,None,YES


,id,money_pot,amount,additional_description,budget_effect_date,creation_date,modification_dates,comments,complete
0,20170802,KG,-568.00,None,None,2017-08-01,None,None,YES
1,20170803,KG,-450.00,None,None,2017-08-02,None,None,YES
2,20170804,KG,-49.05,None,None,2017-08-01,None,None,YES
3,20170805,KG,-7.50,None,None,2017-08-01,None,None,YES
4,20170811,KG,-7.99,None,None,2017-08-10,None,None,YES
5,20170806,KG,-39.80,None,None,2017-08-03,None,None,YES
6,20170732,KG,-23.97,None,None,2017-07-31,None,None,YES
7,20170733,KG,-87.40,None,None,2017-07-31,None,None,YES
8,20170734,KG,-9.44,None,None,2017-07-31,None,None,YES
9,20170807,KG,-8.71,None,None,2017-08-01,None,None,YES


In [20]:
content = get_as_dataframe(sheet, parse_dates=True, skiprows=3, header=1)

In [21]:
display(content)

,Realer Geldverbrauch,Unnamed: 1,Unnamed: 2,=SUM(R[1]C[0]:R[95]C[0]),Unnamed: 4,Unnamed: 5,Unnamed: 6,Bargeld Max,Unnamed: 8,=SUM(R[1]C[0]:R[97]C[0]),Bargeld Paul,Unnamed: 11,=SUM(R[1]C[0]:R[97]C[0]).1,Konto Ing-DiBa,Unnamed: 14,=SUM(R[1]C[0]:R[95]C[0]).1,Unnamed: 16
0,R,Miete,01/08/17,-568,Haben Vermögen,NaN,=R[1]C[0]+R[2]C[0],Aktueller Stand,NaN,"47,2",Aktueller Stand,NaN,NaN,Aktueller Stand,NaN,"405,74",NaN
1,R,Miete FFM,02/08/17,-450,NaN,Verfügung,=R[-4]C[9]+R[-2]C[9]+R[-2]C[6]+R[-2]C[3]+R[-3]...,Geld aufladen Mitarbeiter ausweis,01/08/17,-20,Geld aus Schatulle,NaN,=1,Budgetbeitrag von BMS,28/07/17,1468,NaN
2,R,Berufsunfähigkeitsversicherung,01/08/17,"-49,05",NaN,Sparen,=R[-4]C[9]+R[-6]C[9],Geld abheben,03/08/17,30,Brot,01/08/17,"=-0,95",Aldi,31/07/17,"-23,97",NaN
3,R,Strom EnviaM,NaN,-51,NaN,NaN,NaN,Geld aufladen Trianon,02/08/17,-20,Brot,08/08/17,"-1,4",Monatskarte FFM,31/07/17,"-87,4",NaN
4,R,Vodafone,NaN,"-19,99",Einnahmen,NaN,NaN,Geld aus Schatulle,05/08/17,"1,3",Geld aus Schatulle,08/08/17,"1,35",Netto,31/07/17,"-9,44",NaN
5,M,Telefonie,NaN,-6,Budget-Einnahmen,NaN,NaN,Betriebsausflug,07/08/17,"-18,5",Geld abheben,NaN,20,Haftpflichtversicherung,01/08/17,"-7,5",NaN
6,R,Rechtsschutzversicherung,30/01/17,"-13,9",Sondereinnahmen,NaN,NaN,Geldfund,12/08/17,"0,05",PUCK aufladen,NaN,-20,Berufsunfähigkeitsversicherung,01/08/17,"-49,05",NaN
7,R,Haftpflichtversicherung,01/08/17,"-7,5",Realvermögenszuwachs (1 Mo),NaN,=R[-7]C[0]-Juli!R[-7]C[0],Eisessen,13/08/17,"-4,8",Geld abheben,21/08/17,20,Miete,01/08/17,-568,NaN
8,G,GEW,NaN,"-2,5",Realvermögenszuwachs (3 Mo),NaN,=R[-8]C[0]-'Mai+Juni'!R[-8]C[0],Wasser,13/08/17,"-1,3",NaN,NaN,NaN,Rewe,01/08/17,"-8,71",NaN
9,R,Semestergebühr Paul,21/06/16,"=-276,91/6",NaN,NaN,,BBk Karte Aufladen,14/08/17,-10,NaN,NaN,NaN,Rossmann,01/08/17,"-9,17",NaN


In [34]:
mytry = content.iloc[100].copy()
mytry[0] = 'A'
mytry[1] = 'Eis'
mytry[2] = '25/08/17'
mytry[3] = -1.2
display(mytry)

from numpy import NaN
content.loc[101] = mytry

display(content)

Realer Geldverbrauch                 A
Unnamed: 1                         Eis
Unnamed: 2                    25/08/17
=SUM(R[1]C[0]:R[95]C[0])          -1.2
Unnamed: 4                         NaN
Unnamed: 5                         NaN
Unnamed: 6                         NaN
Bargeld Max                        NaN
Unnamed: 8                         NaN
=SUM(R[1]C[0]:R[97]C[0])           NaN
Bargeld Paul                       NaN
Unnamed: 11                        NaN
=SUM(R[1]C[0]:R[97]C[0]).1         NaN
Konto Ing-DiBa                     NaN
Unnamed: 14                        NaN
=SUM(R[1]C[0]:R[95]C[0]).1         NaN
Unnamed: 16                        NaN
Name: 100, dtype: object

,Realer Geldverbrauch,Unnamed: 1,Unnamed: 2,=SUM(R[1]C[0]:R[95]C[0]),Unnamed: 4,Unnamed: 5,Unnamed: 6,Bargeld Max,Unnamed: 8,=SUM(R[1]C[0]:R[97]C[0]),Bargeld Paul,Unnamed: 11,=SUM(R[1]C[0]:R[97]C[0]).1,Konto Ing-DiBa,Unnamed: 14,=SUM(R[1]C[0]:R[95]C[0]).1,Unnamed: 16
0,R,Miete,01/08/17,-568,Haben Vermögen,NaN,=R[1]C[0]+R[2]C[0],Aktueller Stand,NaN,"47,2",Aktueller Stand,NaN,NaN,Aktueller Stand,NaN,"405,74",NaN
1,R,Miete FFM,02/08/17,-450,NaN,Verfügung,=R[-4]C[9]+R[-2]C[9]+R[-2]C[6]+R[-2]C[3]+R[-3]...,Geld aufladen Mitarbeiter ausweis,01/08/17,-20,Geld aus Schatulle,NaN,=1,Budgetbeitrag von BMS,28/07/17,1468,NaN
2,R,Berufsunfähigkeitsversicherung,01/08/17,"-49,05",NaN,Sparen,=R[-4]C[9]+R[-6]C[9],Geld abheben,03/08/17,30,Brot,01/08/17,"=-0,95",Aldi,31/07/17,"-23,97",NaN
3,R,Strom EnviaM,NaN,-51,NaN,NaN,NaN,Geld aufladen Trianon,02/08/17,-20,Brot,08/08/17,"-1,4",Monatskarte FFM,31/07/17,"-87,4",NaN
4,R,Vodafone,NaN,"-19,99",Einnahmen,NaN,NaN,Geld aus Schatulle,05/08/17,"1,3",Geld aus Schatulle,08/08/17,"1,35",Netto,31/07/17,"-9,44",NaN
5,M,Telefonie,NaN,-6,Budget-Einnahmen,NaN,NaN,Betriebsausflug,07/08/17,"-18,5",Geld abheben,NaN,20,Haftpflichtversicherung,01/08/17,"-7,5",NaN
6,R,Rechtsschutzversicherung,30/01/17,"-13,9",Sondereinnahmen,NaN,NaN,Geldfund,12/08/17,"0,05",PUCK aufladen,NaN,-20,Berufsunfähigkeitsversicherung,01/08/17,"-49,05",NaN
7,R,Haftpflichtversicherung,01/08/17,"-7,5",Realvermögenszuwachs (1 Mo),NaN,=R[-7]C[0]-Juli!R[-7]C[0],Eisessen,13/08/17,"-4,8",Geld abheben,21/08/17,20,Miete,01/08/17,-568,NaN
8,G,GEW,NaN,"-2,5",Realvermögenszuwachs (3 Mo),NaN,=R[-8]C[0]-'Mai+Juni'!R[-8]C[0],Wasser,13/08/17,"-1,3",NaN,NaN,NaN,Rewe,01/08/17,"-8,71",NaN
9,R,Semestergebühr Paul,21/06/16,"=-276,91/6",NaN,NaN,,BBk Karte Aufladen,14/08/17,-10,NaN,NaN,NaN,Rossmann,01/08/17,"-9,17",NaN


In [35]:
new_sheet = connect_to_sheets(workbook, 'new_August')

set_with_dataframe(new_sheet, content)

WorksheetNotFound: new_August

In [37]:
sheet.update_acell('A107', 'A')
sheet.update_acell('B107', 'Eis')
sheet.update_acell('C107', '25/08/17')
sheet.update_acell('D107', -1.2)

In [38]:
sheet.update_acell('D107', '-1,20 €')

In [43]:
first_col = sheet.col_values(1)

In [50]:
print(first_col.index('', 5)) 
## note python list is counted by 0, the excel counting starts at 1, i.e. the result 107 means that row 108 is 
## the first one thats empty
first_col[107]

107


''